# Crop Screenshots Notebook

This notebook is designed to crop JPEG images generated from PNG screenshots. 
Using the Python Imaging Library (Pillow), it processes each JPEG image located in a specified input directory, 
applies a predefined bounding box to crop the images, and saves the cropped versions in a designated output directory. 
The cropping is particularly useful for isolating specific parts of the images, such as selections or highlights, 
to focus on relevant content.

## Features:
- Automatically searches for JPEG images in the input directory.
- Crops images based on specified pixel coordinates for a bounding box.
- Saves the cropped images with a modified filename to differentiate them from the originals.
- Produces console output for each cropped image, indicating conversion success.

Ensure that the input directory contains JPEG images before running the notebook, and modify the 
bounding box dimensions as needed to achieve the desired cropping effect.

In [ ]:
%pip install pillow

In [1]:
# Bounding box (in pixels)
SQUAD_BATTLES_SELECTION_BBOX = (140, 363, 140 + 290, 363 + 545)  # (left, upper, right, lower)

In [ ]:
import os
from PIL import Image

# Input and output directories for image processing
input_dir = '../screenshots/jpg'
output_dir = '../screenshots/jpg/cropped'

# Create the output directory if it doesn't already exist to store cropped images
os.makedirs(output_dir, exist_ok=True)

# Define the cropping box coordinates (left, upper, right, lower) for the crop area
SQUAD_BATTLES_SELECTION_BBOX = (140, 363, 140 + 290, 363 + 545)  # Example crop box

# Loop through all files in the input directory
for file_name in os.listdir(input_dir):
    # Check if the file is a JPG image (case insensitive)
    if file_name.lower().endswith('.jpg'):
        # Construct the full file path for the current image
        jpg_file_path = os.path.join(input_dir, file_name)
        
        # Create the output file name by appending '_cropped' to the base name
        base_name = os.path.splitext(file_name)[0]
        cropped_file_path = os.path.join(output_dir, f"{base_name}_cropped.jpg")
        
        # Open the JPG file using the PIL library
        with Image.open(jpg_file_path) as img:
            # Crop the image using the defined crop box
            cropped_img = img.crop(SQUAD_BATTLES_SELECTION_BBOX)
            # Save the cropped image to the output directory as a JPG with specified quality
            cropped_img.save(cropped_file_path, 'JPEG', quality=90)  # Adjust quality if needed
        
        # Print the file paths of the original and cropped images for reference
        print(f"Cropped: {jpg_file_path} -> {cropped_file_path}")

# Indicate that the cropping process has completed
print("Cropping completed. Check the output directory.")
